<a class="anchor" id="welcome"></a> 
>
>
> <img src="https://i.ibb.co/XWHZ1c4/1.png" alt="Drawing"  style="width: 950px;"/>
><br />
>  <img src="https://i.ibb.co/3zmp6Cd/6767.png" alt="Drawing"  style="width: 950px;"/>
<br />
><br />
><b> 
> <span style='font-family:Verdana '> Msc in Business Analytics @ AEUB  <br />
>  <span style='font-family:Verdana '> Course: Mining Big Datasets – Assignment #1  <br />
>  <span style='font-family:Verdana '> Professor: Y.Kotidis (kotidis@aueb.gr) <br />
>  <span style='font-family:Verdana '> Assistant: I.Filippidou (filippidoui@aueb.gr) <br />
>  <span style='font-family:Verdana '> Student 1: Despotis Spyridon | ID: P2822111 <br />
>  <span style='font-family:Verdana '> Student 2: Papailiou Thanasis | ID: P2822128  <br />
>  <span style='font-family:Verdana '> Date: 24/05/2022 <br /></b> 

***

> <img src="https://i.ibb.co/Y0z9Qv5/2.png" alt="Drawing" style="width: 380px;"/> <br />
><br />
> 00 - [Introduction](#introduction) <br />
> 01 - [Importing and Pre-processing](#question1) <br />
> 02 - [Computing data (dis-)similarity](#question2) <br />
> 03 - [Nearest Neighbor (NN) search](#question3) <br />
> 04 - [Customer rating prediction](#question4) <br />
> 05 - [Conclusions](#question5) <br />
***

<a class="anchor" id="introduction"></a> 
> <img src="https://i.ibb.co/SNNSQym/3.png" alt="Drawing" style="width: 460px;"/><br /> 
> <br />
>  <br />
> The goal of this assignment is to implement a simple workflow that will assess the 
similarity between bank customers and suggest for any input customer a list of his/her
10 most similar other customers. Moreover, we will be using these results to predict 
the rating of the customer to the bank. To calculate the similarity between customers
you will first have to compute the dissimilarity for every given attribute as discussed
in lecture “Measuring Data Similarity”. 
<br />

> Description of the available attributes: <br />
> <b> Age</b> : The age of the customer. <br />
> <b> Job</b> : type of job (admin, unknown, unemployed, management, housemaid, entrepreneur, student, blue-collar, self-employed, retired, technician, services). <br />
> <b> Marital Status </b> : Married, Single, Divorced. <br />
> <b>  Education </b> : Primary, Secondary, Tertiary. <br />
> <b> Default </b> : If the customer has credit in default (yes/no). <br />
> <b> Balance </b> : average yearly balance, in euros. <br />
> <b> Housing </b> : if the customer has a housing loan (yes/no). <br />
> <b> Loan </b> : if the customer has a personal loan (yes/no) <br />
> <b> Customer Rating </b> : The rating of the bank from the customer (Poor, Fair, Good, Very Good, Excellent). <br />
> <b> Products </b> : An array containing the bank products (1-20) each customer has. <br />
***



<a class="anchor" id="question1"></a> 
<img src="https://i.ibb.co/HXXdN0Y/first.png" alt="Drawing" style="width: 960px;"/><br /> 
 <br />

<b> <span style="font-size: 200%"> <span style='font-family:Helvetica'> Let's Start our Analysis! </b> 

* Due to the fact that the downloaded dataset is csv data type, we used read_csv function
* We run as many different commands we can in order to have a straight forward view of our dataset

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
pip install IPython

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install termcolor

Note: you may need to restart the kernel to use updated packages.


In [4]:
from IPython.display import display
from termcolor import colored
import numpy as np
import pandas as pd

df = pd.read_csv("bank.csv", sep = ";", header = 0)
df.index = df.index + 1
display(df.head())
display(df.describe())
display(df.info())


,Age,Job,Marital,Education,Default,Balance,Housing,Loan,Rating,Products
1,33.0,entrepreneur,married,secondary,no,2,yes,yes,poor,"1,3,16,17,19"
2,35.0,management,married,tertiary,no,231,yes,no,good,"4,8,16"
3,NaN,management,single,tertiary,no,447,yes,yes,fair,"7,16"
4,42.0,entrepreneur,divorced,tertiary,yes,2,yes,no,fair,"1,3,8,10,11,12,18,19"
5,58.0,retired,married,primary,no,121,yes,no,good,"4,5,6,7,11,18,19"


,Age,Balance
count,42191.000000,43191.000000
mean,40.792065,1354.039754
std,10.540425,3042.165010
min,18.000000,-8019.000000
25%,33.000000,71.000000
50%,39.000000,442.000000
75%,48.000000,1412.000000
max,95.000000,102127.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43191 entries, 1 to 43191
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Age        42191 non-null  float64
 1   Job        43191 non-null  object 
 2   Marital    43191 non-null  object 
 3   Education  43191 non-null  object 
 4   Default    43191 non-null  object 
 5   Balance    43191 non-null  int64  
 6   Housing    43191 non-null  object 
 7   Loan       43191 non-null  object 
 8   Rating     43191 non-null  object 
 9   Products   43191 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 3.3+ MB


None

# Conversions

* We convert the data columns in the appropriate data types

In [5]:
df = df.astype({
    "Age": float, 
    "Job": str, 
    "Marital": str,      
    "Education": str, 
    "Default": bool, 
    "Balance": float,
    "Housing": bool,
    "Loan": bool,
    "Rating": str,
    "Products":str})

# Ordinal Data

* Note that Education & Rating are ordinal variables, i.e. a categorical variable with an inherent order to it. pandas has not listed the values of that variable in the order we would expect (low, lower-middle, upper-middle, high). 
* The order of a categorical variable can be specified in the dataframe, using the top level pandas function Categorical().



In [6]:
df['Education'].unique()

array(['secondary', 'tertiary', 'primary'], dtype=object)

In [7]:
df['Rating'].unique()

array(['poor', 'good', 'fair', 'very_good', 'excelent'], dtype=object)

In [8]:
df["Education"] = (pd.Categorical(df["Education"], ordered=True,categories=["primary", "secondary", "tertiary"]))
df["Rating"] = (pd.Categorical(df["Rating"], ordered=True, categories=[ "poor","fair","good","very_good","excelent"]))
df["Job"] = pd.Categorical(df["Job"])
df["Marital"] = pd.Categorical(df["Marital"])

In [9]:
df

,Age,Job,Marital,Education,Default,Balance,Housing,Loan,Rating,Products
1,33.0,entrepreneur,married,secondary,True,2.0,True,True,poor,"1,3,16,17,19"
2,35.0,management,married,tertiary,True,231.0,True,True,good,"4,8,16"
3,NaN,management,single,tertiary,True,447.0,True,True,fair,"7,16"
4,42.0,entrepreneur,divorced,tertiary,True,2.0,True,True,fair,"1,3,8,10,11,12,18,19"
5,58.0,retired,married,primary,True,121.0,True,True,good,"4,5,6,7,11,18,19"
...,...,...,...,...,...,...,...,...,...,...
43187,51.0,technician,married,tertiary,True,825.0,True,True,very_good,"12,15"
43188,71.0,retired,divorced,primary,True,1729.0,True,True,very_good,"4,17,19"
43189,72.0,retired,married,secondary,True,5715.0,True,True,very_good,"5,7,8,9,10,11,13,16,17,18"
43190,57.0,blue-collar,married,secondary,True,668.0,True,True,very_good,"2,3,7"


* Next we transform the column Products to a list

In [10]:
df["Products"] = df["Products"].apply(lambda x: set(x.split(",")))

# Missing Values

* Next we check for missing values

In [11]:
print("Number of empty values in each columns:")
df.isnull().sum()

Number of empty values in each columns:


Age          1000
Job             0
Marital         0
Education       0
Default         0
Balance         0
Housing         0
Loan            0
Rating          0
Products        0
dtype: int64

* As we can see in the age column we have many null values
* We will fill them with the average value of the attribute (rounded to the nearest integer)
* Last but not least, now that we dont have any NA's we convert column Age to interger

In [12]:
df["Age"] = df["Age"].fillna(np.int32(round(df["Age"].mean(skipna=True))))
df.isnull().sum()

Age          0
Job          0
Marital      0
Education    0
Default      0
Balance      0
Housing      0
Loan         0
Rating       0
Products     0
dtype: int64

# Checking the Final Structure

In [13]:
df.head(100)

,Age,Job,Marital,Education,Default,Balance,Housing,Loan,Rating,Products
1,33.0,entrepreneur,married,secondary,True,2.0,True,True,poor,"{16, 17, 3, 1, 19}"
2,35.0,management,married,tertiary,True,231.0,True,True,good,"{8, 4, 16}"
3,41.0,management,single,tertiary,True,447.0,True,True,fair,"{16, 7}"
4,42.0,entrepreneur,divorced,tertiary,True,2.0,True,True,fair,"{18, 8, 11, 12, 3, 1, 19, 10}"
5,58.0,retired,married,primary,True,121.0,True,True,good,"{4, 18, 11, 6, 5, 7, 19}"
...,...,...,...,...,...,...,...,...,...,...
96,34.0,admin.,married,secondary,True,61.0,True,True,good,"{4, 14, 18, 2, 13, 3, 5, 1, 10}"
97,59.0,retired,single,secondary,True,30.0,True,True,good,"{9, 8, 2, 12, 1}"
98,45.0,management,married,tertiary,True,523.0,True,True,good,"{11, 17, 14, 12}"
99,41.0,services,divorced,secondary,True,31.0,True,True,good,{12}


***

<a class="anchor" id="question2"></a> 
<img src="https://i.ibb.co/4gmP83y/2-data-disimilarity.png" alt="Drawing" style="width: 990px;"/><br /> 
 <br />

# The Methodology

In this section, we calculate the similarity / dis-similarity (1-similarity) between the
customers. Given the size of the dataset, we decided it was not feasible to calculate and
store the dissimilarity matrix in memory. 

On the other hand, doing the dissimilarity calculations on-the-fly proved to be sufficiently fast. 
For this goal, we implemented a set of functions, one for each attribute type. All functions accept the same arguments, the specific attribute’s value for a customer, and the column containing the same attributes
for all customers. The dissimilarity for all customers is returned.

In order to compute the total similarity (or dissimilarity), we compute the metric for
each column and we take the average of all values. In the subsections below we describe
the method used to calculate dissimilarity for each attribute type we encountered.

# Formulas

* We have to compute the similarity according to the data type of each attribute.
* For example, for set similarity we will use Jacard similarity
* Bellow are all the measures for disimilarity of each column type


# 1. <b>Categorical</b>  <br /> 
* The ordinal variables take the absolute difference between the rank of the two customers, divided by the difference of of maximum rank and minimum rank. 
<br /><img src="https://i.ibb.co/GTbYs7x/ordinal-removebg-preview.png" alt="Drawing" style="width: 290px;"/><br /> 
<br />
* For unordered variables, the disimilarity is 0 if the value is the same, 1 otherwise.

In [14]:
def discategorical(a, b):

    if not b.cat.ordered: return 1 - (b == a).astype(int)

    a = (b.cat.categories == a).argmax()
    
    diss = (a - b.cat.codes).abs() / (b.cat.codes.max() - b.cat.codes.min())
    return diss

# 2. <b>Nominal</b><br />   
The nominal variables will have value "1" if they have no similarity betweeen the values of the two customers, and "0" if they have the same values.<br /><br />

In [15]:
def disnominal(a, b):
    
    diss = 1 - (b == a).astype(int)
    return diss

# 3.<b> Numeric</b>  <br /> 
The numeric variables take the absolute difference between the two customers, divided by the difference of maximum and minimum numeric values. 
<br /><img src="https://i.ibb.co/Jc1VQXw/numeric-removebg-preview.png" alt="Drawing" style="width: 290px;"/><br />
<br />

In [16]:
def disnumerical(a, b):
    
    diss = abs(b - a) / (b.max() - b.min())
    return diss

# 4. <b>Set</b> <br />
For set of variables we calculate Jacard similarity. The Jaccard similarity index (sometimes called the Jaccard similarity coefficient) compares members for two sets to see which members are shared and which are distinct. It’s a measure of similarity for the two sets of data, with a range from 0% to 100%. The higher the percentage, the more similar the two populations.<br /> <img src="https://i.ibb.co/55Y8ffX/jacard-removebg-preview.png" alt="Drawing" style="width: 290px;"/><br />

In [17]:
def disset(a1, b: pd.Series):
    diss = 1 - b.apply(lambda a2: (len(a1.intersection(a2)) / len(a1.union(a2))))
    return diss

We have created the above function, which for a given dataframe `df`, it calculates the dissimilarity between the customer with id `id` and all other customers.

The arguments are the following
- `df`: The used dataset
- `id`: The customer ID to compare to all other customers

For each column, we execute the appropriate disimilarity function and store the results
in a dataframe, where each column corresponds to one variable and the values are the disimilarities.

The total disimilarity is equal to the average values across columns.

In [18]:
dis = {
    "bool": disnominal,
    "int64": disnumerical,
    "float64": disnumerical,
    "category": discategorical,
    "object": disset,
}

def calcdis(df: pd.DataFrame, id: int, ignore_cols=[]):

    row = df.loc[id]

    df_dis = pd.concat(
        [
            dis[dtype.name](row[col], df[col]) for col, dtype in df.dtypes.iteritems() if col not in ignore_cols
        ],
        axis=1,
    )

    return df_dis.mean(axis=1)

***

<a class="anchor" id="question3"></a> 
<img src="https://i.ibb.co/jDgBSvb/nearest-neighbor.png" alt="Drawing" style="width: 990px;"/><br /> 
 <br />

# The Methodology & Process

* To find the nearest neighbours of a customer, we begin by utilising the functions prepared in step 2 to calculate the dissimilarity between a customer and all others. 
* The customers are then sorted by ascending dissimilarity in order to output the 10 most similar ones.

In [19]:
def nn(df: pd.DataFrame, id: int, ignore_cols=[]):
    sim = 1 - calcdis(df, id, ignore_cols)
    sim = sim.loc[sim.index != id]
    return sim.sort_values(ascending=False).head(10)

# The Results

* Below we calculate the NNs for the requested customers ( 1200, 3650, 10400, 14930, 22330, 25671, 29311, 34650, 39200, 42000 ).

In [20]:
print()
display("Most Similar Customers Results")
for target in [1200, 3650, 10400, 14930, 22330, 25671, 29311, 34650, 39200, 42000]:
    nneighbors = nn(df, target)
    print("───────────────────────────────────")
    print(colored(f"Nearest neighbors for customer {target}:" ,'cyan',attrs=['bold','reverse']))
    display(nneighbors.reset_index(drop=False).rename(columns={"index": "Customer ID", 0: "Similarity"}) )

'Most Similar Customers Results'

───────────────────────────────────
Nearest neighbors for customer 1200:


,Customer ID,Similarity
0,41726,0.954603
1,20870,0.932426
2,14733,0.932350
3,38856,0.928460
4,41664,0.926438
5,16371,0.924450
6,23961,0.923867
7,24833,0.923335
8,16983,0.923040
9,8756,0.920661


───────────────────────────────────
Nearest neighbors for customer 3650:


,Customer ID,Similarity
0,6189,0.950560
1,28141,0.937784
2,8067,0.936680
3,6018,0.936171
4,5616,0.927949
5,7307,0.923025
6,33844,0.922993
7,20262,0.918398
8,33475,0.916447
9,7713,0.914521


...

,Customer ID,Similarity
0,35426,0.935788
1,15291,0.935487
2,6902,0.931291
3,692,0.930733
4,43075,0.929214
5,27643,0.927901
6,7828,0.926375
7,20567,0.924065
8,7658,0.923906
9,13318,0.923095


───────────────────────────────────
Nearest neighbors for customer 39200:


,Customer ID,Similarity
0,13171,0.958222
1,1889,0.938777
2,26044,0.936109
3,29359,0.929250
4,37141,0.928357
5,13047,0.922020
6,9744,0.921712
7,29463,0.921251
8,36358,0.921039
9,27083,0.920309


───────────────────────────────────
Nearest neighbors for customer 42000:


,Customer ID,Similarity
0,32471,0.958657
1,28027,0.950954
2,31801,0.943429
3,41316,0.942950
4,39518,0.937329
5,22742,0.935395
6,41144,0.935216
7,42419,0.935024
8,32493,0.933916
9,39425,0.933633


***

<a class="anchor" id="question4"></a> 
<img src="https://i.ibb.co/C6tzSwP/customer-rating.png" alt="Drawing" style="width: 990px;"/><br /> 
 <br />

# The Methodology

* In this section we implemented a classification algorithm for a customer in order to predict his rating (poor, fair, good, very good and excellent) for the bank. <br />
* In order to achieve that, first we had to calculate the similarities between the customer and all other customers and compute his 10-nn most similar customers <br />
* Then based only on the 10 most similar customers we computed in the previous step, we predicted the customer rating rank using:<br />
1) The average rating rank of the 10 most similar customers (rounded to the nearest integer). <br />
2) The weighted average rating rank of the 10 most similar customers (rounded to the nearest integer).<br /> 
 <img src="https://i.ibb.co/2MYLskj/weighted-rank.png" alt="Drawing" style="width: 590px;"/><br /> 
 Where:<br /> 
 i) rating(i) = the rating of the i-th nearest neighbor (i=1 for the most similar customer)<br />  
 ii) similarity(i) = the similarity of the i-th nearest neighbor with the given customer<br /> 
3) For the evaluation of your classification algorithm we used the 50 first records of the bank dataset and predicted the rating for them. Then, for all n=50 records we calculated the Mean Prediction Error for both prediction methods.<br /> 
 <img src="https://i.ibb.co/7yt7ZTg/mean-prediction-error.png" alt="Drawing" style="width: 590px;"/><br /> 

# The Process

* We create a new dataframe which is a copy from the first "df" dataframe
* The "np.nan" is the IEEE 754 floating-point representation of Not a Number. The nan stands for “not a number“, and its primary constant is to act as a placeholder for any missing numerical values in the array.
* The "cat.categories"  converts the Rating column into category type.

In [21]:
prediction_df = df.head(50).copy()
prediction_df["Predicted_Rating"] = np.nan
prediction_df["Weighted_Predicted_Rating"] = np.nan
ratings = df["Rating"].cat.categories
display(prediction_df.head(15))
display(ratings)

,Age,Job,Marital,Education,Default,Balance,Housing,Loan,Rating,Products,Predicted_Rating,Weighted_Predicted_Rating
1,33.0,entrepreneur,married,secondary,True,2.0,True,True,poor,"{16, 17, 3, 1, 19}",NaN,NaN
2,35.0,management,married,tertiary,True,231.0,True,True,good,"{8, 4, 16}",NaN,NaN
3,41.0,management,single,tertiary,True,447.0,True,True,fair,"{16, 7}",NaN,NaN
4,42.0,entrepreneur,divorced,tertiary,True,2.0,True,True,fair,"{18, 8, 11, 12, 3, 1, 19, 10}",NaN,NaN
5,58.0,retired,married,primary,True,121.0,True,True,good,"{4, 18, 11, 6, 5, 7, 19}",NaN,NaN
6,43.0,technician,single,secondary,True,593.0,True,True,good,"{8, 11, 16, 17, 20, 5, 7, 1, 19}",NaN,NaN
7,41.0,admin.,divorced,secondary,True,270.0,True,True,good,"{4, 14, 9, 11, 2, 17, 12, 3, 5, 10}",NaN,NaN
8,29.0,admin.,single,secondary,True,390.0,True,True,good,{17},NaN,NaN
9,53.0,technician,married,secondary,True,6.0,True,True,good,"{18, 14, 8, 16, 17, 20, 13, 19}",NaN,NaN
10,41.0,services,married,secondary,True,162.0,True,True,good,{3},NaN,NaN


Index(['poor', 'fair', 'good', 'very_good', 'excelent'], dtype='object')

* The bellow function finds the 10NNs most similar customers
* It ignores the default index
* Then calculates the predicted rating for this customer
* And assigns the results to a dataframe

In [22]:
for i, row in prediction_df.iterrows():
        nneighbors = nn(df, i, ignore_cols=["Rating"])
        df_nn = df.loc[nneighbors.index].copy()
        df_nn["Similarity"] = nneighbors
        df_nn["Rating_code"] = df_nn["Rating"].cat.codes
        df_nn["Weighted_rating_code"] = df_nn["Similarity"] * df_nn["Rating_code"]
        rating = df_nn["Rating_code"].mean(skipna=True)
        rating = ratings[int(rating.round())]
        prediction_df.loc[i, "Predicted_Rating"] = rating
        weighted_rating = (
            df_nn["Weighted_rating_code"].sum() / df_nn["Similarity"].sum())
        weighted_rating = ratings[
            int(weighted_rating.round())
        ]  
        prediction_df.loc[i, "Weighted_Predicted_Rating"] = weighted_rating

* Bellow we can see the predictions for the 50 first records of the bank dataset

In [23]:
display("Customer rating predictions:")
display((prediction_df.loc[:, ["Rating", "Predicted_Rating", "Weighted_Predicted_Rating"]]))

'Customer rating predictions:'

,Rating,Predicted_Rating,Weighted_Predicted_Rating
1,poor,good,good
2,good,good,good
3,fair,good,good
4,fair,good,good
5,good,good,good
6,good,good,good
7,good,good,good
8,good,good,good
9,good,good,good
10,good,good,fair


* Next we convert rating columns to categorical in order to make comparisons with the true values

In [24]:
prediction_df["Predicted_Rating"] = pd.Categorical(prediction_df["Predicted_Rating"],categories=[ "poor","fair", "good","very_good", "excelent", ],ordered=True,)
prediction_df["Weighted_Predicted_Rating"] = pd.Categorical(prediction_df["Weighted_Predicted_Rating"], categories=["poor", "fair","good","very_good","excelent",],ordered=True,)
prediction_df.head(50)

,Age,Job,Marital,Education,Default,Balance,Housing,Loan,Rating,Products,Predicted_Rating,Weighted_Predicted_Rating
1,33.0,entrepreneur,married,secondary,True,2.0,True,True,poor,"{16, 17, 3, 1, 19}",good,good
2,35.0,management,married,tertiary,True,231.0,True,True,good,"{8, 4, 16}",good,good
3,41.0,management,single,tertiary,True,447.0,True,True,fair,"{16, 7}",good,good
4,42.0,entrepreneur,divorced,tertiary,True,2.0,True,True,fair,"{18, 8, 11, 12, 3, 1, 19, 10}",good,good
5,58.0,retired,married,primary,True,121.0,True,True,good,"{4, 18, 11, 6, 5, 7, 19}",good,good
6,43.0,technician,single,secondary,True,593.0,True,True,good,"{8, 11, 16, 17, 20, 5, 7, 1, 19}",good,good
7,41.0,admin.,divorced,secondary,True,270.0,True,True,good,"{4, 14, 9, 11, 2, 17, 12, 3, 5, 10}",good,good
8,29.0,admin.,single,secondary,True,390.0,True,True,good,{17},good,good
9,53.0,technician,married,secondary,True,6.0,True,True,good,"{18, 14, 8, 16, 17, 20, 13, 19}",good,good
10,41.0,services,married,secondary,True,162.0,True,True,good,{3},good,fair


* Finally we calculated the Mean Prediction Error (refers to the average difference between the model's predicted values and the actual values).

In [25]:
mse = (prediction_df["Predicted_Rating"].cat.codes - prediction_df["Rating"].cat.codes).abs().sum() / len(prediction_df)
mse_weighted = (prediction_df["Weighted_Predicted_Rating"].cat.codes- prediction_df["Rating"].cat.codes).abs().sum() / len(prediction_df)
print(f"Mean Prediction Error (average method): {mse}")
print(f"Mean Prediction Error (weighted average method): {mse_weighted}")

Mean Prediction Error (average method): 0.56
Mean Prediction Error (weighted average method): 0.62


* From the Mean Prediction Error we can conclude that the average method has lower error than the weighted method.
* Both methods have very similar results in prediction error terms
* Average Method: The average deviation between the predicted points scored and the actual points scored is 0.56
* Weighted Average Method: The average deviation between the predicted points scored and the actual points scored is 0.62

***

<a class="anchor" id="question5"></a> 
<img src="https://i.ibb.co/BsF9vKF/conlusions.png" alt="Drawing" style="width: 490px;"/><br /> 
 <br />
 <img src="https://i.ibb.co/ScNfJt3/s90-td-12778-01-mockup.jpg" alt="Drawing"  style="width: 750px;"/>

* This assignment was a greate change to study and a experiment with a real scenario in Data Similarity
* We started with data cleaning and transformation
* Then we applied many different formulas and measures of (dis)simillarity for different data types
* Also, we learned more about the automation of the process by creating new functions for finding nearest neighbours
* Last but not least, we had a straight forward view in prediction method

***

[Scroll To Top ^](#welcome) <img style="float: right;" src="https://i.ibb.co/w7Lbd0X/2-AUEB-white-HR.jpg"  alt="aueb" width="550px"/>